**Datacamp Project**

# Analyzing River Thames Water Levels
Time series data is everywhere, from watching your stock portfolio to monitoring climate change, and even live-tracking as local cases of a virus become a global pandemic. In this project, you’ll work with a time series that tracks the tide levels of the Thames River. You’ll first load the data and inspect it data visually, and then perform calculations on the dataset to generate some summary statistics. You’ll end by reducing the time series to its component attributes and analyzing them. 

The original dataset is available from the British Oceanographic Data Center.

Here's a map of the locations of the tidal meters along the River Thames in London.

![](locations.png)

The provided datasets are in the `data` folder in this workspace. For this project, you will work with one of these files, `10-11_London_Bridge.txt`, which contains comma separated values for water levels in the Thames River at the London Bridge. After you've finished the project, you can use your same code to analyze data from the other files (at other spots in the UK where tidal data is collected) if you'd like. 

The TXT file contains data for three variables, described in the table below. 

| Variable Name | Description | Format |
| ------------- | ----------- | ------ |
| Date and time | Date and time of measurement to GMT. Note the tide gauge is accurate to one minute. | dd/mm/yyyy hh:mm:ss |
| Water level | High or low water level measured by tide meter. Tide gauges are accurate to 1 centimetre. | metres (Admiralty Chart Datum (CD), Ordnance Datum Newlyn (ODN or Trinity High Water (THW)) | 
| Flag | High water flag = 1, low water flag = 0 | Categorical (0 or 1) |



In [27]:
#0. Importing pandas package and defining IQR function
import pandas as pd               

def IQR(column): 
    """ Calculates the interquartile range (IQR) for a given DataFrame column using the quantile method """
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

#1. Loading and filtering data from London Bridge

#Reading text file with pandas
data = pd.read_csv("data/10-11_London_Bridge.txt")
data.head()

#Working with the first 3 columns of the dataframe, since the last one isn't necessary for the analysis
data = data.iloc[:, :3]
data.head()

#Renaming columns for easier manipulation
data.columns = ['datetime', 'water_level', 'high_tide']
data.head()

,datetime,water_level,high_tide
0,01/05/1911 15:40:00,3.7130,1
1,02/05/1911 11:25:00,-2.9415,0
2,02/05/1911 16:05:00,3.3828,1
3,03/05/1911 11:50:00,-2.6367,0
4,03/05/1911 16:55:00,2.9256,1


In [28]:
#2. Preparing data columns for analysis

#Converting Date and time column to datetime data type
data['datetime'] = pd.to_datetime(data['datetime'])
data.head()

#Converting Water level column to float data type
data['water_level'] = data.water_level.astype(float)
data.head()

#Extracting month and year from datetime and creating new columns for easy access
data['month'] = data['datetime'].dt.month
data['year'] = data['datetime'].dt.year
data.head()

,datetime,water_level,high_tide,month,year
0,1911-01-05 15:40:00,3.7130,1,1,1911
1,1911-02-05 11:25:00,-2.9415,0,2,1911
2,1911-02-05 16:05:00,3.3828,1,2,1911
3,1911-03-05 11:50:00,-2.6367,0,3,1911
4,1911-03-05 16:55:00,2.9256,1,3,1911


In [29]:
#3. Separating dataframe for high and low tide water levels

#Filtering dataframe for each water level value
high_tide = data[data['high_tide'] == 1]
high_tide.head()

low_tide = data[data['high_tide'] == 0]
low_tide.head()

#4. Finding summary statistics for new dataframes - mean, median and IQR

#Creating 2 separate pandas series for summary statistics
high_stats = high_tide['water_level'].agg(['mean', 'median', IQR])
low_stats = low_tide['water_level'].agg(['mean', 'median', IQR])
print(high_stats)
print(low_stats)

mean      3.318373
median    3.352600
IQR       0.743600
Name: water_level, dtype: float64
mean     -2.383737
median   -2.412900
IQR       0.538200
Name: water_level, dtype: float64


In [30]:
#5. Determining percentages of high and low tide level days each year

#Calculating ratio for high tide days
all_high_days = high_tide.groupby('year')['water_level'].count()
very_high_days = high_tide[high_tide['water_level'] > high_tide['water_level'].quantile(0.90)].groupby('year')['water_level'].count()
very_high_ratio = (very_high_days/all_high_days).reset_index()
very_high_ratio

#Calculating ratio for low tide days
all_low_days = low_tide.groupby('year')['water_level'].count()
very_low_days = low_tide[low_tide['water_level'] < low_tide['water_level'].quantile(0.10)].groupby('year')['water_level'].count()
very_low_ratio = (very_low_days/all_low_days).reset_index()
very_low_ratio

#6. Creating solution dictionary
solution = {'high_statistics': high_stats, 'low_statistics': low_stats, 'very_high_ratio': very_high_ratio, 'very_low_ratio': very_low_ratio}
print(solution)

{'high_statistics': mean      3.318373
median    3.352600
IQR       0.743600
Name: water_level, dtype: float64, 'low_statistics': mean     -2.383737
median   -2.412900
IQR       0.538200
Name: water_level, dtype: float64, 'very_high_ratio':     year  water_level
0   1911     0.004098
1   1912     0.032316
2   1913     0.082212
3   1914     0.055313
4   1915     0.045045
..   ...          ...
80  1991     0.096317
81  1992     0.103253
82  1993     0.145923
83  1994     0.150355
84  1995     0.170213

[85 rows x 2 columns], 'very_low_ratio':     year  water_level
0   1911     0.060606
1   1912     0.066667
2   1913     0.022388
3   1914     0.039017
4   1915     0.033435
..   ...          ...
80  1991     0.150355
81  1992     0.107496
82  1993     0.112696
83  1994     0.106383
84  1995     0.107801

[85 rows x 2 columns]}
